# Label image for YOLOv8s

In [5]:
import cv2
import numpy as np
import os
import shutil

# Root folders
input_root = "C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL/test_data/images/u2net_results"
output_root = "C:/Users/admin/Documents/GitHub/Pill_Identification/Pill_YOLO_Labels"
output_boundary_root = "C:/Users/admin/Documents/GitHub/Pill_Identification/Pill_YOLO_bounding_boxes"

# Remove output folders if they exist
if os.path.exists(output_root):
    shutil.rmtree(output_root)
if os.path.exists(output_boundary_root):
    shutil.rmtree(output_boundary_root)

# Supported image extensions
image_extensions = ('.png', '.jpg', '.jpeg')

for root, dirs, files in os.walk(input_root):
    for file in files:
        if file.lower().endswith(image_extensions):
            input_path = os.path.join(root, file)

            # Build relative path and new output paths
            relative_path = os.path.relpath(input_path, input_root)
            relative_dir = os.path.dirname(relative_path)
            filename_wo_ext = os.path.splitext(file)[0]

            # Output .txt and bbox image paths
            txt_output_path = os.path.join(output_root, relative_dir, filename_wo_ext + ".txt")
            bbox_output_path = os.path.join(output_boundary_root, relative_dir, filename_wo_ext + "_bbox.png")

            # Create output directories
            os.makedirs(os.path.dirname(txt_output_path), exist_ok=True)
            os.makedirs(os.path.dirname(bbox_output_path), exist_ok=True)

            # Load mask
            mask = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
            if mask is None:
                print(f"❌ Failed to load image: {input_path}")
                continue

            # Threshold to binary
            _, binary = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

            # Find contours
            contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            height, width = binary.shape
            output_img = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

            label_lines = []

            # Extract class ID from filename prefix
            class_id = filename_wo_ext.split('_')[0]

            for cnt in contours:
                x, y, w, h = cv2.boundingRect(cnt)

                # Convert to YOLO format
                x_center = (x + w / 2) / width
                y_center = (y + h / 2) / height
                w_norm = w / width
                h_norm = h / height

                yolo_format = f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}"
                label_lines.append(yolo_format)

                # Draw bounding box
                cv2.rectangle(output_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Save YOLO label file
            with open(txt_output_path, "w") as f:
                for line in label_lines:
                    f.write(line)  # Add newline per line

            # Save image with bounding boxes
            cv2.imwrite(bbox_output_path, output_img)
            print(f"✅ Processed: {input_path}")


✅ Processed: C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL/test_data/images/u2net_results\0_MS_01.png
✅ Processed: C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL/test_data/images/u2net_results\0_MS_02.png
✅ Processed: C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL/test_data/images/u2net_results\0_MT_01.png
✅ Processed: C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL/test_data/images/u2net_results\0_MT_02.png
✅ Processed: C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL/test_data/images/u2net_results\0_MT_03.png
✅ Processed: C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL/test_data/images/u2net_results\10_MS_101.png
✅ Processed: C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL/test_data/images/u2net_results\10_MS_102.png
✅ Processed: C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL

# Label image for RetinaNet

In [10]:
import cv2
import numpy as np
import os
import shutil
import csv

# Root folders
input_root = "C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL/test_data/images/u2net_results"
csv_output_path = "C:/Users/admin/Documents/GitHub/Pill_Identification/pill_bounding_boxes.csv"

# Remove existing CSV
if os.path.exists(csv_output_path):
    os.remove(csv_output_path)

# Supported image extensions
image_extensions = ('.png', '.jpg', '.jpeg')

# Open CSV writer
with open(csv_output_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['image_id', 'width', 'height', 'class_id', 'x', 'y', 'w', 'h'])

    for root, dirs, files in os.walk(input_root):
        for file in files:
            if file.lower().endswith(image_extensions):
                input_path = os.path.join(root, file)
                filename_wo_ext = os.path.splitext(file)[0]
                image_id = filename_wo_ext

                # Load mask
                mask = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
                if mask is None:
                    print(f"❌ Failed to load image: {input_path}")
                    continue

                # Threshold to binary
                _, binary = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

                # Find contours
                contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                height, width = binary.shape

                # Extract class ID from filename (e.g., 0_MS_01.png → class_id = 0)
                class_id = filename_wo_ext.split('_')[0]

                for cnt in contours:
                    x, y, w, h = cv2.boundingRect(cnt)

                    # Convert to float
                    x_f = float(x)
                    y_f = float(y)
                    w_f = float(w)
                    h_f = float(h)

                    # Write row
                    csv_writer.writerow([image_id, width, height, class_id, x_f, y_f, w_f, h_f])

                print(f"✅ Processed: {input_path}")


✅ Processed: C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL/test_data/images/u2net_results\0_MS_01.png
✅ Processed: C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL/test_data/images/u2net_results\0_MS_02.png
✅ Processed: C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL/test_data/images/u2net_results\0_MT_01.png
✅ Processed: C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL/test_data/images/u2net_results\0_MT_02.png
✅ Processed: C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL/test_data/images/u2net_results\0_MT_03.png
✅ Processed: C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL/test_data/images/u2net_results\10_MS_101.png
✅ Processed: C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL/test_data/images/u2net_results\10_MS_102.png
✅ Processed: C:/Users/admin/Documents/GitHub/Pill_Identification/background_removal_DL